In [99]:
from project_functions_final_df import *
import pandas as pd
import geopandas as gpd
import numpy as np
import os 
from pathlib import Path  
import matplotlib.pyplot as plt

# Variable Assignments

In [62]:
clienttoken = "MLY|6271859919532158|486bcdece7a665eb89a5803f2994b4c9"
bounds = [(12.436781, 55.545432), (12.436781, 55.731815), (12.696966, 55.731815), (12.696966, 55.545432)]
my_shapefile_path = "koebenhavn.shp"
my_shp = gpd.read_file(my_shapefile_path)

In [79]:
ids_file = pd.read_csv('ids.csv')
IDs =  ids_file['image_id'].to_list()
np.random.seed(42)
IDs = list(np.random.choice(IDs, 1000000))

In [80]:
strava_csv = pd.read_csv("/home/s204124/7f42d69dd7bd00b04081b9cbd0d05ab4e5f87353dcd7364c34b8e7c74d42109d-1676891223740.csv")
df_activity = StravaToDF(my_shp, bounds, strava_csv)

# Create dataset

### Get Detections for Mapillary

In [76]:
df_detections, dict_image = GetDetections_FaF(IDs, clienttoken, data_type = 'int')

Loading…: 100%|████████████████████████| 1000/1000 [03:29<00:00,  4.77it/s]


### Export Detections Dataframe

In [66]:
current_path = os.getcwd()
filepath = Path(current_path+'/detections_df.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_detections.to_csv(filepath) 

### Create Model Data

In [78]:
pip = GetPointsInPoly(my_shp, clienttoken, bounds, IDs)
model_data = DFTransform(df_detections, pip, df_activity, dict_image, threshold = 0.005)

             image_id  object--bench  object--trash-can  \
0    1008435179879344            1.0                7.0   
1    1016476232215216            NaN                NaN   
2    1020563291813865            1.0                2.0   
3    1030251200716472            NaN                NaN   
4     103425619305174            NaN                NaN   
..                ...            ...                ...   
724   967954973972922            NaN                NaN   
725   969726210437953            NaN                NaN   
726   978081352998926            NaN                NaN   
727   983575275958307            NaN                NaN   
728   991604554913960            NaN                NaN   

     construction--barrier--curb  construction--flat--bike-lane  \
0                           17.0                            NaN   
1                            NaN                            NaN   
2                            NaN                            NaN   
3                      

Loading…: 100%|████████████████████████| 1000/1000 [01:28<00:00, 11.24it/s]
/home/s204124/project_functions_final7.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_polygons["geometry"] = df_polygons["geometry"].buffer(buffer, cap_style = 2)


                                             detection  occurence
0    marking--discrete--arrow--split-right-or-straight          1
1                                         nature--sand          1
2                            regulatory--turn-left--g1          1
3                      complementary--chevron-left--g5          1
4                    information--minimum-speed-30--g1          1
..                                                 ...        ...
148                                 nature--vegetation        206
149                           construction--flat--road        208
150                                        nature--sky        211
151                              object--support--pole        232
152                               object--street-light        346

[153 rows x 2 columns]
         edge_id  object--bench  marking--discrete--stop-line  \
0    174399596.0            0.0                           0.0   
1    174399698.0            0.0                       

### Reassign Activity to Classes

In [72]:
ReassignActivity(model_data)

### Export Model Data

In [73]:
current_path = os.getcwd()
filepath = Path(current_path+'/modeldata_final.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
model_data.to_csv(filepath)